# **kcBert-base**
- 학습률 스케쥴러



In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 데이터 로드

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/GBT 해커톤/data/train_df_1012.csv')
test_df = pd.read_csv('/content/drive/MyDrive/GBT 해커톤/data/test_df_1012.csv')

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54314 entries, 0 to 54313
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54314 non-null  object
 1   분류      54314 non-null  object
 2   제목      54314 non-null  object
 3   키워드     54314 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   제목      23405 non-null  object
 2   키워드     23405 non-null  object
dtypes: object(3)
memory usage: 548.7+ KB


### 모델링

- epoch: 10
- learning rate: 5e-5
- batch size: 32
- max length: 256

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from transformers import get_linear_schedule_with_warmup

In [ ]:
config = {
    "learning_rate": 5e-5,
    "epoch": 10,
    "batch_size": 32
}

CFG = SimpleNamespace(**config)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('beomi/KcBERT-base')
model = BertForSequenceClassification.from_pretrained('beomi/KcBERT-base', num_labels=len(train_df['분류'].unique())).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item] if self.labels is not None else -1
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.키워드.tolist(), train_df.label.tolist(), tokenizer)
val_dataset = TextDataset(val_df.키워드.tolist(), val_df.label.tolist(), tokenizer)
test_dataset = TextDataset(test_df.키워드.tolist(), None, tokenizer)  # 라벨 없음

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [ ]:
train_dataset[0]

{'text': '세비야시 스페인 도시 교환 우호 교류 의향서 경제 이상일 문화 협력 도시 발전 스페인 세비야 경제 문화 협력 스페인 방문 이상일 현지시간 세비야 시청 호세 루이스 산즈 세비야 도시 우호교류 도시 우호 교류 향서 교환 경제 문화 관광 분야 교류 협력 세비야 주도 스페인 남서부 안달루시아 세비야 주의 알카사르 궁전 세비야 유네스코 세계 문화유산 이베로 아메리칸 엑스포 스페인광장 관광자원 보유 기준 세계 관광객 도시 만큼 관광 세비야 국내총생산 안달루시아 지방 총생산 GDP 차지 산업 중심지 섬유 자동차 제조업 금융 발달 신재 에너지 항공 우주 산업 분야 경쟁력 도시 우호교류 향서 안정적 우호 협력 강화 경제 대표단 교류 박람회 세미나 개최 문화 예술단 교류 홍보 경제 문화 관광 교육 환경 분야 발전 소통 협력 안달루시아 진주 세비야 교류 협력 국민 세비야 방면 교류 도시 것들 발전 산즈 세비야 교류 공동발전 도모 방문 검토 우호 교류 향서 교환 공약 민선 도시 세계 미래 산업 추진 자매 우호 결연 이행 차원 이달 미국 텍사스주 윌리엄슨 카운티 우호교류 향서 교환 윌리엄슨 카운티 삼성전자 투자 반도체 테일러 포함',
 'input_ids': tensor([    2, 18387,  4144,  4039,  1983, 24158, 11860,   341,  4185, 22509,
         20679,  2442,  4573,  4072,  8065,  8176,  4046, 10398, 12898, 11860,
          9578,  1983, 24158, 18387,  4144,  8065, 10398, 12898,  1983, 24158,
         12669,  8176,  4046, 21864,  8432, 18387,  4144, 11585,  3443,  4066,
          1235, 12198,  1789,  4146, 18387,  4144, 11860, 22509,  4267,  4316,
        

In [ ]:
# 옵티마이저 및 학습 파라미터 설정
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * CFG.epoch)

In [13]:
# 초기화
patience = 2  # 개선되지 않을 경우 기다리는 에폭 수
best_f1 = 0.0  # 최상의 F1 스코어 초기화
counter = 0  # 카운터 초기화

for epoch in range(CFG.epoch):
    model.train()

    # 학습 단계
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())

    # F1 스코어 계산
    current_f1 = f1_score(val_true_labels, val_predictions, average='macro')

    # Early stopping 체크
    if current_f1 > best_f1:
        best_f1 = current_f1  # 최상의 F1 스코어 갱신
        counter = 0  # 카운터 초기화
        torch.save(model.state_dict(), f'model_best_f1.pth')  # 모델 저장
        print(f"Model saved with F1 Score: {best_f1:.4f}")
    else:
        counter += 1  # 카운터 증가

    print(f"Epoch {epoch + 1}, F1 Score: {current_f1:.4f}")

    # Early stopping이 활성화되면 훈련 종료
    if counter >= patience:
        print("Early stopping triggered. Training stopped.")
        break

Validating: 100%|██████████| 340/340 [02:54<00:00,  1.95it/s]


Model saved with F1 Score: 0.4889
Epoch 1, F1 Score: 0.4889


Validating: 100%|██████████| 340/340 [02:54<00:00,  1.94it/s]


Model saved with F1 Score: 0.5688
Epoch 2, F1 Score: 0.5688


Validating: 100%|██████████| 340/340 [02:54<00:00,  1.95it/s]


...

Model saved with F1 Score: 0.6674
Epoch 8, F1 Score: 0.6674


Validating: 100%|██████████| 340/340 [02:54<00:00,  1.94it/s]


Model saved with F1 Score: 0.6693
Epoch 9, F1 Score: 0.6693


Validating: 100%|██████████| 340/340 [02:54<00:00,  1.94it/s]


Model saved with F1 Score: 0.6703
Epoch 10, F1 Score: 0.6703


In [14]:
# 테스트 세트 추론
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

Testing: 100%|██████████| 732/732 [06:15<00:00,  1.95it/s]


In [ ]:
sample_submission = pd.read_csv("/content/drive/MyDrive/GBT 해커톤/data/sample_submission.csv")
sample_submission["분류"] = decoded_predictions

sample_submission.to_csv("/content/drive/MyDrive/GBT 해커톤/data/submission_KcBert-base_1015.csv", encoding='UTF-8-sig', index=False)

In [16]:
result = pd.read_csv("/content/drive/MyDrive/GBT 해커톤/data/submission_KcBert-base_1015.csv")
result.head()

,ID,분류
0,TEST_00000,지역
1,TEST_00001,사회:의료_건강
2,TEST_00002,정치:행정_자치
3,TEST_00003,경제:취업_창업
4,TEST_00004,지역


In [17]:
result['분류'].value_counts()

,count
분류,
지역,11807
경제:부동산,1479
사회:사건_사고,1149
경제:반도체,1003
사회:사회일반,595
사회:의료_건강,449
사회:교육_시험,423
정치:국회_정당,411
스포츠:올림픽_아시안게임,353


,count
분류,
지역,11807
경제:부동산,1479
사회:사건_사고,1149
경제:반도체,1003
사회:사회일반,595
사회:의료_건강,449
사회:교육_시험,423
정치:국회_정당,411
스포츠:올림픽_아시안게임,353
